In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

sample_submission_v2.csv
test-jpg-v2
test-tif-v3
test_v2_file_mapping.csv
train-jpg
train-tif-v2
train_v2.csv



In [2]:
train = pd.read_csv('../input/train_v2.csv')
test = pd.read_csv('../input/sample_submission_v2.csv')

In [3]:
labels = train['tags'].apply(lambda x: x.split(' '))

In [4]:
label_list=[]
for i in labels:
    for j in i:
        if j not in label_list:
            label_list.append(j)

In [5]:
x_train = []
x_test = []
y_train = []

In [6]:
label_map = {l: i for i, l in enumerate(label_list)}

In [7]:
label_map

{'agriculture': 2,
 'artisinal_mine': 13,
 'bare_ground': 12,
 'blooming': 14,
 'blow_down': 16,
 'clear': 3,
 'cloudy': 9,
 'conventional_mine': 11,
 'cultivation': 7,
 'habitation': 5,
 'haze': 0,
 'partly_cloudy': 10,
 'primary': 1,
 'road': 6,
 'selective_logging': 15,
 'slash_burn': 8,
 'water': 4}

In [8]:
from tqdm import tqdm
import cv2

In [9]:
for f, tags in tqdm(train.values, miniters=1000):
    img = cv2.imread('../input/train-jpg/{}.jpg'.format(f))
    img = cv2.resize(img,(64,64))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    x_train.append(img)
    y_train.append(targets)

100%|██████████| 40479/40479 [01:20<00:00, 501.74it/s]


In [10]:
for f, tags in tqdm(test.values, miniters=1000):
    img = cv2.imread('../input/test-jpg-v2/{}.jpg'.format(f))
    img = cv2.resize(img,(64,64))
    x_test.append(cv2.resize(img,(64, 64)))

100%|██████████| 61191/61191 [02:06<00:00, 485.59it/s]


In [11]:
y_train = np.array(y_train)
x_train = np.array(x_train, np.float32)/255.0
x_test  = np.array(x_test, np.float32)/255.0

In [12]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(40479, 64, 64, 3)
(61191, 64, 64, 3)
(40479, 17)


In [13]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
np.random.seed(1671)

Using TensorFlow backend.


In [14]:
model = Sequential()
model.add(Conv2D(32, (3,3), strides=(1,1), padding='same', input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(64, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3), strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same', strides=(2,2)))
model.add(Conv2D(128, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3,3),  padding='same' ,strides=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same', strides=(2,2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(17))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_3 (Activation)    (None, 32, 32, 64)        0         
__________

In [15]:
OPTIMIZER = Adam( lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=OPTIMIZER , metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=64, epochs=2, verbose=1, validation_split=0.25)

Train on 30359 samples, validate on 10120 samples
Epoch 1/2
30359/30359 [==============================] - 831s - loss: 0.3403 - acc: 0.8392 - val_loss: 0.3092 - val_acc: 0.8334
Epoch 2/2
30359/30359 [==============================] - 786s - loss: 0.3032 - acc: 0.8442 - val_loss: 0.2915 - val_acc: 0.8493


In [16]:
OPTIMIZER.lr = 0.0001
history = model.fit(x_train, y_train, batch_size=64, epochs=2, verbose=1, validation_split=0.25)

Train on 30359 samples, validate on 10120 samples
Epoch 1/2
30359/30359 [==============================] - 821s - loss: 0.2921 - acc: 0.8452 - val_loss: 0.2828 - val_acc: 0.8501
Epoch 2/2
30359/30359 [==============================] - 968s - loss: 0.2858 - acc: 0.8452 - val_loss: 0.2831 - val_acc: 0.8446


In [17]:
y_test=[]
p_test = model.predict(x_test)
y_test.append(p_test)

In [20]:
p_test.shape

(61191, 17)

In [23]:
p_test

array([[  1.52585201e-03,   4.76469904e-01,   3.54401604e-03, ...,
          3.72456945e-03,   7.20482087e-04,   3.31533927e-04],
       [  7.27720093e-04,   5.04245102e-01,   2.77534630e-02, ...,
          8.50166101e-03,   6.35497412e-03,   1.93678390e-03],
       [  1.95105576e-06,   4.28624719e-01,   4.75963280e-02, ...,
          1.30684668e-07,   2.60895445e-06,   4.73363834e-05],
       ..., 
       [  2.61066307e-04,   4.78923887e-01,   2.73500849e-03, ...,
          3.35305464e-03,   6.06619171e-04,   2.09104051e-04],
       [  2.12481230e-01,   2.19007835e-01,   3.05098966e-02, ...,
          6.61704180e-05,   1.03656370e-04,   1.76394969e-05],
       [  2.71117012e-03,   1.74869612e-01,   1.56395108e-01, ...,
          2.37577115e-05,   2.85016053e-04,   5.92635515e-05]], dtype=float32)

In [27]:
y_test[0].shape

(61191, 17)

In [32]:
result = pd.DataFrame(p_test, columns = label_list)
result

,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,0.001526,0.476470,0.003544,0.495546,0.013804,0.000270,0.001609,0.001509,4.107288e-05,0.000098,0.000658,2.631743e-06,0.000141,3.886498e-06,3.724569e-03,7.204821e-04,0.000332
1,0.000728,0.504245,0.027753,0.349496,0.044598,0.001972,0.008457,0.023630,8.708615e-04,0.000069,0.020730,1.779296e-05,0.000601,3.898905e-05,8.501661e-03,6.354974e-03,0.001937
2,0.000002,0.428625,0.047596,0.000025,0.039340,0.001782,0.019612,0.006864,4.489973e-06,0.001669,0.454347,2.912217e-06,0.000028,5.326875e-05,1.306847e-07,2.608954e-06,0.000047
3,0.001561,0.468221,0.016482,0.451416,0.033571,0.000880,0.005943,0.010302,1.980971e-04,0.000072,0.002397,6.848387e-06,0.000514,1.320809e-05,4.994913e-03,2.560020e-03,0.000868
4,0.000952,0.393877,0.036397,0.000168,0.030395,0.003321,0.024382,0.004643,5.075958e-06,0.131154,0.374498,1.053115e-05,0.000142,5.006572e-05,1.064098e-06,5.755866e-07,0.000003
5,0.000172,0.476843,0.001734,0.509535,0.006889,0.000113,0.000726,0.000766,1.453954e-05,0.000011,0.000183,3.811229e-07,0.000033,6.992310e-07,2.493731e-03,3.637229e-04,0.000122
6,0.019850,0.268644,0.149369,0.099066,0.083599,0.025182,0.060357,0.107691,1.303747e-02,0.001003,0.130021,9.740561e-04,0.009417,1.085493e-03,9.354495e-03,1.506124e-02,0.006289
7,0.000301,0.195717,0.171090,0.224634,0.032044,0.123299,0.232072,0.010607,1.505694e-04,0.000011,0.000633,3.838474e-04,0.008523,4.718367e-04,1.094202e-06,5.825342e-05,0.000003
8,0.000621,0.477316,0.002527,0.502152,0.010237,0.000181,0.001113,0.001094,2.658117e-05,0.000043,0.000413,1.190234e-06,0.000076,1.885550e-06,3.409072e-03,5.556333e-04,0.000232
9,0.174380,0.363571,0.096277,0.190358,0.072492,0.008413,0.041344,0.028648,8.251120e-04,0.004308,0.012979,1.814090e-04,0.003785,1.956425e-04,6.282422e-04,1.080472e-03,0.000532


In [34]:
from tqdm import tqdm
preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.ix[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

100%|██████████| 61191/61191 [02:40<00:00, 380.10it/s]


In [35]:
preds

['primary clear',
 'primary clear',
 'primary partly_cloudy',
 'primary clear',
 'primary partly_cloudy',
 'primary clear',
 'primary',
 'clear road',
 'primary clear',
 'primary',
 'primary partly_cloudy',
 'primary clear',
 'primary cloudy',
 'primary clear',
 'primary clear',
 'primary clear',
 'primary agriculture clear',
 'primary clear',
 'primary clear',
 'primary clear',
 'primary clear',
 'primary clear',
 'primary agriculture clear',
 'cloudy',
 'primary partly_cloudy',
 'primary clear',
 'primary',
 'primary agriculture clear',
 'primary',
 'primary clear',
 'primary partly_cloudy',
 'primary cloudy',
 'primary clear',
 'primary agriculture clear',
 'cloudy',
 'primary clear',
 'primary clear',
 'primary clear',
 'primary clear',
 'primary clear',
 'primary clear water',
 'primary partly_cloudy',
 'primary clear',
 'primary clear',
 'primary clear',
 'primary',
 'primary',
 'primary clear',
 'primary clear',
 'primary',
 'primary clear',
 'primary clear',
 'primary agricultu

In [36]:
test['tags'] = preds
test.to_csv('submission.csv', index=False)

In [52]:
test.to_csv('sample_submission_v2.csv', index=False)

In [55]:
test.to_csv('sample_submission_v2.csv', index=False)

In [57]:
test.to_csv('resul.csv', index=False)

In [54]:
k = pd.read_csv('sample_submission_v2.csv')
k.head()

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary clear
4,test_4,primary partly_cloudy


In [62]:
print(check_output(["ls", "./"]).decode("utf8"))

__notebook_source__.ipynb
resul.csv
sample_submission_v2.csv
submission.csv



In [63]:
ls

__notebook_source__.ipynb  resul.csv  sample_submission_v2.csv  submission.csv


In [68]:
pwd

'/tmp'